In [1]:
import numpy as np
import random
from scipy.spatial import distance
import time

In [2]:
def merge(database_seq, query_seq, remove_f):
    #ti = time.time()
    N_d = database_seq.shape[0]
    N_q = query_seq.shape[0]
    _data = database_seq.tolist()
    temp = query_seq.tolist()
    
    for i in range(N_q-1,-1,-1):
        if i % remove_f == 0:
            del temp[i]
        else:
            temp[i][2] = 0
    #tf1 = time.time()
    #print("1:",tf1-ti)
    data = list()
    
    x_i = random.randint(0,19)
    y_i = random.randint(0,9)
    x_f = x_i + 140
    y_f = y_i + 110
    
    for item in _data:
        if (item[0] >= x_i and item[0] < x_f) and (item[1] >= y_i and item[1] < y_f):
            item[0] -= x_i
            item[1] -= y_i
            data.append(item)
    #tf2 = time.time()
    #print("2:",tf2-tf1)
    x_i = random.randint(0,19)
    y_i = random.randint(0,9)
    x_f = x_i + 140
    y_f = y_i + 110
        
    for item in temp:
        if (item[0] >= x_i and item[0] < x_f) and (item[1] >= y_i and item[1] < y_f):
            item[0] -= x_i
            item[1] -= y_i
            data.append(item)
    #tf3 = time.time()
    #print("3:",tf3-tf2)
    data = sorted(data, key = lambda x : x[3])
    
    #tf4 = time.time()
    #print("4:",tf4-tf3)
    return data



In [3]:
env = "clean"
date = "2020_07_04_11_24_02"
ms = 100
threshold = 10 #m

database_file = "../dataset/KH/clean/raw_numpy_"+str(ms)+"ms/"
database_utm_file = open("../dataset/KH/"+env+"/raw_numpy_"+str(ms)+"ms/"+date+"_location.txt",'r')
output_location = "/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_"+str(ms)+"ms_syn/"

ground_file = open(output_location+"train.txt",'w')

n_true = 0
n_false = 0
D_utm = list()

while True:
    line = database_utm_file.readline()
    if not line:
        break
    line_list = line.split()
    item = [int(line_list[0]),float(line_list[1]),float(line_list[2])]
    D_utm.append(item)
database_utm_file.close()


for rm_f in range(2,6):
    S = set()
    while n_true < 50000 or n_false < 50000:
        #ti = time.time()
        d_num = random.randint(0,len(D_utm)-1)
        q_num = random.randint(0,len(D_utm)-1)
        database_np = np.load(database_file + date + '/' + str(d_num) + ".npy")
        query_np = np.load(database_file + date + '/' + str(q_num) + ".npy")

        if (d_num, q_num) in S:
            continue
        #if rain_file.shape[0] > T:
        #    continue
        d_file = np.load(database_file + date + '/' + str(d_num) + ".npy")
        q_file = np.load(database_file + date + '/' + str(q_num) + ".npy")

        utm_d = (D_utm[d_num][1], D_utm[d_num][2])
        utm_q = (D_utm[q_num][1], D_utm[q_num][2])
        d = distance.euclidean(utm_d, utm_q)
        if d <= threshold:
            ground_file.write(str(rm_f)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+"\t1\n")
            n_true += 1
        else:
            if n_false >= n_true:
                continue
            ground_file.write(str(rm_f)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+"\t0\n")
            n_false += 1
        #tf = time.time()
        #print("find:", tf-ti)
        S.add((d_num, q_num))
        result = merge(d_file, q_file, rm_f)
        np.save(output_location+str(rm_f)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+".npy",result)
    n_true = 0
    n_false = 0
ground_file.close()

In [4]:
def merge_2(database_seq, query_seq):
    #ti = time.time()
    N_d = database_seq.shape[0]
    N_q = query_seq.shape[0]
    _data = database_seq.tolist()
    temp = query_seq.tolist()
    
    #tf1 = time.time()
    #print("1:",tf1-ti)
    data = list()
    
    x_i = 10
    y_i = 0
    x_f = x_i + 140
    y_f = y_i + 110
    
    for item in _data:
        if (item[0] >= x_i and item[0] < x_f) and (item[1] >= y_i and item[1] < y_f):
            item[0] -= x_i
            item[1] -= y_i
            data.append(item)
    #tf2 = time.time()
    #print("2:",tf2-tf1)
    x_i = 10
    y_i = 5
    x_f = x_i + 140
    y_f = y_i + 110
        
    for item in temp:
        if (item[0] >= x_i and item[0] < x_f) and (item[1] >= y_i and item[1] < y_f):
            item[0] -= x_i
            item[1] -= y_i
            data.append(item)
    #tf3 = time.time()
    #print("3:",tf3-tf2)
    data = sorted(data, key = lambda x : x[3])
    
    #tf4 = time.time()
    #print("4:",tf4-tf3)
    return data


In [12]:
env_q = "rain"
date_q = "2020_07_30_17_51_53"

query_file = "../dataset/KH/"+env_q+"/raw_numpy_"+str(ms)+"ms/"
query_utm_file = open("../dataset/KH/"+env_q+"/raw_numpy_"+str(ms)+"ms/"+date_q+"_location.txt",'r')
output_location = "/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_"+str(ms)+"ms_syn/"
ground_file = open(output_location+"test.txt",'w')
n_true_test = 0
n_false_test = 0
Q_utm = list()

while True:
    line = query_utm_file.readline()
    if not line:
        break
    line_list = line.split()
    item = [int(line_list[0]),float(line_list[1]),float(line_list[2])]
    Q_utm.append(item)
query_utm_file.close()

S = set()

while n_true_test < 100 or n_false_test < 100:
    d_num = random.randint(0,len(D_utm)-1)
    q_num = random.randint(0,len(Q_utm)-1)
    database_np = np.load(database_file + date + '/' + str(d_num) + ".npy")
    query_np = np.load(query_file + date_q + '/' + str(q_num) + ".npy")

    if (d_num, q_num) in S:
        continue
    #if rain_file.shape[0] > T:
    #    continue
    d_file = np.load(database_file + date + '/' + str(d_num) + ".npy")
    q_file = np.load(query_file + date_q + '/' + str(q_num) + ".npy")

    utm_d = (D_utm[d_num][1], D_utm[d_num][2])
    utm_q = (Q_utm[q_num][1], Q_utm[q_num][2])
    d = distance.euclidean(utm_d, utm_q)
    if d <= threshold:
        ground_file.write(str(6)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+"\t1\n")
        n_true_test += 1
    else:
        if n_false_test >= n_true_test:
            continue
        ground_file.write(str(6)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+"\t0\n")
        n_false_test += 1
    #tf = time.time()
    #print("find:", tf-ti)
    S.add((d_num, q_num))
    result = merge_2(d_file, q_file)
    np.save(output_location+str(6)+'0'*(4-len(str(d_num)))+str(d_num)+'0'+'0'*(4-len(str(q_num)))+str(q_num)+".npy",result)
ground_file.close()